In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '2'
import sys
from pathlib import Path
import anndata as ad
import scanpy as sc
import numpy as np
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import seaborn as sns
from matplotlib import pyplot as plt
sys.path.insert(0, "/home/ylu/project/spateo-release")
import spateo as st
from spateo.alignment.methods.morpho_class import *
# sys.path.insert(0, "/home/ylu/project/spateo-release_new")
# import spateo_dev as st_dev
sys.path.insert(0, "/home/ylu/project")
from utils import *
import time


%load_ext autoreload
%autoreload 2

2024-07-17 05:29:49.209520: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-17 05:29:49.370194: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-12.3/lib64
2024-07-17 05:29:49.370223: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2024-07-17 05:29:50.072129: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot ope

In [2]:
data_folder = "../../data/STARMapPlus/"
method = 'Spateo'
i = 3
adata = ad.read_h5ad(os.path.join(data_folder, f"sagittal{i}.h5ad"))
scale = np.linalg.norm(adata.obsm['spatial'].max(0) - adata.obsm['spatial'].min(0))
slices = split_slice(
    adata=adata,
    spatial_key='spatial_raw',
    split_num=2,
    axis=2,
)

In [3]:
overlay_ratio_set = np.arange(0.5, 1.05, 0.05)
from tqdm import tqdm
maes = []
for overlay_ratio in tqdm(overlay_ratio_set):
    slice1 = slices[0].copy()
    slice2 = slices[1].copy()
    slice1_crop, slice2_crop = crop_slices(
        slice1, 
        slice2, 
        spatial_key='spatial', 
        overlay_ratio = overlay_ratio,
    )
    slice1, slice2 = slice1_crop.copy(), slice2_crop.copy()
    sampling_num = 20000
    sampline_idx1 = np.random.choice(slice1.shape[0], sampling_num, replace=False) if slice1.shape[0] > sampling_num else np.arange(slice1.shape[0])
    sampline_idx2 = np.random.choice(slice2.shape[0], sampling_num, replace=False) if slice2.shape[0] > sampling_num else np.arange(slice2.shape[0])
    slice1 = slice1[sampline_idx1,:]
    slice2 = slice2[sampline_idx2,:]
    spatial_key = 'spatial'
    key_added = 'align_spatial'

    adata_pca = ad.concat([slice1,slice2], label='batch')
    PCA(adata_pca, 'X', 'X_pca', 50, 'cuda')
    slice1.obsm['X_pca'] = adata_pca[adata_pca.obs['batch'] == '0'].obsm['X_pca'].copy()
    slice2.obsm['X_pca'] = adata_pca[adata_pca.obs['batch'] == '1'].obsm['X_pca'].copy()
    
    align_slices = [slice1, slice2]
    morpho_model = Morpho_pairwise(
        sampleA=align_slices[1],
        sampleB=align_slices[0],
        spatial_key=spatial_key,
        key_added=key_added,
        iter_key_added='align_spatial_iter',
        partial_robust_level=50,
        lambdaVF=1e3,
        beta=1e-3,
        sigma2_init_scale=1,
        device='1',
        sparse_calculation_mode=True,
        use_chunk=True,
        chunk_capacity=2,
        rep_layer=['X_pca'],
        rep_field=['obsm'],
        dissimilarity=['cos'],
        nn_init=True,
        nn_init_weight=1,
    )
    P = morpho_model.run()
    align_slices[1].obsm[key_added] = morpho_model.optimal_RnA.copy()
    align_slices[0].obsm[key_added] = align_slices[0].obsm[spatial_key].copy()

    spatial_align = align_slices[1].obsm[key_added]
    spatial_GT = align_slices[1].obsm['spatial']
    mae, mae_var = MAE(spatial_align, spatial_GT) / scale
    maes.append(mae)
np.save(f'./results/{method}_different_overlap.npy', {'maes': maes, 'overlay_ratio':overlay_ratio}, allow_pickle=True)

  0%|                                      | 0/11 [00:00<?, ?it/s]

|-----> Filtered all samples for common genes. There are 1022 common genes.
|-----> Spatial coordinates normalization params:
|-----------> Scale: tensor([16289.2031, 16289.2031])...


/tmp/ipykernel_3221860/1000336554.py:24: ImplicitModificationWarning:

Setting element `.obsm['X_pca']` of view, initializing view as actual.

/tmp/ipykernel_3221860/1000336554.py:25: ImplicitModificationWarning:

Setting element `.obsm['X_pca']` of view, initializing view as actual.



|-----------> Scale: tensor([[17945.0879, 21752.3984],
        [21593.9004, 49982.0625]], device='cuda:0')...
here
|-----> Preprocess finished.
|-----> Performing coarse rigid alignment...
|-----> Coarse rigid alignment done.
|-----> [Start Spateo pairwise alignment] in progress: 100.0000%
|-----> [Start Spateo pairwise alignment] finished [5.4632s]
|-----> Key Parameters: gamma: 0.32312503457069397; sigma2: 0.0010000000474974513; probability_parameters: [tensor(0.0025, device='cuda:0')]


  9%|██▋                           | 1/11 [00:11<01:51, 11.19s/it]

...

 91%|██████████████████████████▎  | 10/11 [01:38<00:09,  9.58s/it]

|-----> Filtered all samples for common genes. There are 1022 common genes.
|-----> Spatial coordinates normalization params:
|-----------> Scale: tensor([22774.9980, 22774.9980])...
|-----------> Scale: tensor([[18291.1387, 33129.4727],
        [21177.1348, 41051.1875]], device='cuda:0')...
here
|-----> Preprocess finished.
|-----> Performing coarse rigid alignment...


/tmp/ipykernel_3221860/1000336554.py:24: ImplicitModificationWarning:

Setting element `.obsm['X_pca']` of view, initializing view as actual.

/tmp/ipykernel_3221860/1000336554.py:25: ImplicitModificationWarning:

Setting element `.obsm['X_pca']` of view, initializing view as actual.



|-----> Coarse rigid alignment done.
|-----> [Start Spateo pairwise alignment] in progress: 100.0000%
|-----> [Start Spateo pairwise alignment] finished [5.8837s]
|-----> Key Parameters: gamma: 0.9728068113327026; sigma2: 0.0041472758166491985; probability_parameters: [tensor(0.0026, device='cuda:0')]


100%|█████████████████████████████| 11/11 [01:47<00:00,  9.81s/it]
